In [240]:
import pandas as pd 
import config
import plotly.express as px
import plotly.offline as pyo
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
from sklearn.feature_selection import SelectFromModel
pd.set_option('display.max_rows', 500)

In [438]:
df = pd.read_csv("/Users/tonypark/Desktop/machine-learning-housing-project/communal/Ames_Housing_Price_Data_cleaned_3.csv")
pd.set_option('display.max_columns', None)
px.set_mapbox_access_token("pk.eyJ1IjoidG9ueXBhcmswMDEiLCJhIjoiY2t2dWVramFmODZqaDJucXBrbnhpZ2JreCJ9.-XXf1YI8YESgWbWODhZAZA")

In [439]:
df.drop("lot_bucket", axis=1, inplace = True)
df.drop("Prop_Addr", axis=1, inplace = True)
df.drop("mean_LotFrontage", axis=1, inplace = True)

In [440]:
df["GarageYrBlt"][df["GarageYrBlt"] == 'None'] = np.nan
df["GarageYrBlt"] = df["GarageYrBlt"].astype("float")

#if garage year build is na, replace with when house was built
#df["GarageYrBlt"]=df["GarageYrBlt"].fillna(df["YearBuilt"])
df.drop("GarageYrBlt", inplace=True, axis=1)

/Users/tonypark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Lasso without Feature Engineering

In [224]:
df["log_SalePrice"] = np.log(df["SalePrice"])

X = df.drop(["SalePrice","PID", "log_SalePrice"], axis=1)
y = df["log_SalePrice"]

In [357]:
#dummify X
X = pd.get_dummies(X, drop_first=True)

In [358]:
#min max scale the variables 
xscaler = MinMaxScaler()
x_train = xscaler.fit_transform(x_train)
x_test = xscaler.transform(x_test)

In [359]:
#test train split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [263]:
clf = linear_model.Lasso()

parameters = {'alpha':np.linspace(0.00001, 1000, 100), 
             }
gs = GridSearchCV(
    linear_model.Lasso(max_iter=1000), 
    parameters, 
    verbose= 3, 
    cv = 5,
    scoring='r2'
)


In [275]:
testr2 = []
trainr2 = []
clf = linear_model.Lasso(max_iter=2000)
for alpha in np.linspace(0.00001, 1000, 100):
    clf.set_params(alpha= alpha)
    clf.fit(x_train, y_train)
    trainr2.append(clf.score(x_train, y_train))
    testr2.append(clf.score(x_test, y_test))


/Users/tonypark/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:646: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.390e+00, tolerance: 2.769e-02



In [ ]:
list(zip(list(gs.best_estimator_.coef_), list(X.columns)))

In [565]:
##MSE 
mean_squared_error(gs.predict(xscaler.transform(x_test)), y_test)

/Users/tonypark/anaconda3/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



9232246759.990479

In [279]:
##get feature importance 
co  = list(zip(list(gs.best_estimator_.coef_), list(X.columns)))

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

## Feature Cleaning & Engineering

In [441]:
#drop PID 
df.drop("PID", axis=1, inplace=True)

In [442]:
#remove outliers 
df.drop([115, 132, 141, 335, 490, 538, 2047, 2239], inplace=True) 

In [410]:
# #categorize neighborhoods 
# df =df.replace("Landmrk", "OldTown")
# df =df.replace("GrnHill", "OldTown")
# df =df.replace("NoRidge", "Somerst")
# df =df.replace("StoneBr", "Somerst")
# df =df.replace("NWAmes", "Gilbert")
# df =df.replace("Veenker", "Somerst")
# df =df.replace("NridgHt", "Somerst")
# df =df.replace("Blmngtn", "Gilbert")
# df =df.replace("Sawyer", "OldTown")
# df =df.replace("Edwards", "OldTown")
# df =df.replace("BrkSide", "OldTown")
# df =df.replace("SWISU", "OldTown")
# df =df.replace("CollgCr", "SawyerW")
# df =df.replace("Crawfor", "Timber")
# df =df.replace("Blueste", "OldTown")
# df =df.replace("ClearCr", "SawyerW")
# df =df.replace("IDOTRR", "OldTown")
# df =df.replace("BrDale", "NAmes")
# df =df.replace("NPkVill", "NAmes")
# df =df.replace("Greens", "Somerst")
# df =df.replace("MeadowV", "OldTown")

In [411]:
# #convert MoSold to object to be dummified 
# df["MoSold"]=df["MoSold"].astype("str")
# df.drop("YrSold", inplace=True, axis=1)

In [443]:
#identify categorical vs numeric columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns = list(df.select_dtypes(include=numerics).columns)
categorical_columns =list(df.drop(numeric_columns, axis=1).columns)


In [444]:
#number of bathrooms
df["Num_of_Bath"] = df["BsmtFullBath"] + 0.5* df["BsmtHalfBath"] + df["FullBath"] + 0.5*df["HalfBath"]
df.drop(["FullBath", 'BsmtFullBath',"BsmtHalfBath","HalfBath"], axis=1, inplace=True)


In [445]:
#drop lat and long 
df.drop(["lat","long"], axis=1, inplace=True)

#has a second floor
# df["has2ndfloor"] = df["2ndFlrSF"].apply(lambda x: 1 if x>0 else 0)

#has a pool 

#has a garage
# df["hasGarage"] = df["GarageArea"].apply(lambda x: 1 if x>0 else 0)

#has a basement 
# df["hasBmt"] = df["TotalBsmtSF"].apply(lambda x: 1 if x>0 else 0)

#has a fireplace
# df["hasFirepalce"] = df["Fireplaces"].apply(lambda x: 1 if x>0 else 0)


#take the log of grLivARea
df["log_GrLivArea"] = np.log(df["GrLivArea"])
df.drop("GrLivArea", axis=1, inplace=True)

#total_living_SF
# df["square_footage_1"] = df["GrLivArea"] + df["TotalBsmtSF"] + df["GarageArea"]

#OverallQual * OverallCond 
# df["total_cond"] = df["OverallQual"] * df["OverallCond"]
# df.drop(["OverallQual","OverallCond"], axis=1, inplace=True)


#dropping all square footage that add up to grlivearea
# df.drop(["1stFlrSF", "2ndFlrSF", "BsmtFinSF1","BsmtFinSF2", "BsmtUnfSF"], inplace=True, axis=1)
# df.drop(["BsmtFinSF1","BsmtFinSF2", "BsmtUnfSF"], inplace=True, axis=1)


In [345]:
##ordinal categorie
# dict_1 = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2,'Po':1, 'None':0}
# dict_2 = {'Gd':4, 'Av':3, 'Mn':2, 'No':1,'None':0}
# dict_3 = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'None':0}
# df["ExterQual"] = df["ExterQual"].apply(lambda x: dict_1[x])
# df["ExterCond"] = df["ExterCond"].apply(lambda x: dict_1[x])
# df["BsmtQual"] = df["BsmtQual"].apply(lambda x: dict_1[x])
# df["BsmtCond"] = df["BsmtCond"].apply(lambda x: dict_1[x])
# df["BsmtExposure"] = df["BsmtExposure"].apply(lambda x: dict_2[x])
# df["BsmtFinType1"] = df["BsmtFinType1"].apply(lambda x: dict_3[x])
# df["HeatingQC"] = df["HeatingQC"].apply(lambda x: dict_1[x])
# df["KitchenQual"] = df["KitchenQual"].apply(lambda x: dict_1[x])
# df["FireplaceQu"] = df["FireplaceQu"].apply(lambda x: dict_1[x])
# df["GarageQual"] = df["GarageQual"].apply(lambda x: dict_1[x])
# df["GarageCond"] = df["GarageCond"].apply(lambda x: dict_1[x])



## Lasso With Features Reduced and Engineered 

In [504]:
df["log_SalePrice"] = np.log(df["SalePrice"])

In [505]:
y = df["log_SalePrice"]

In [506]:
X = df.drop(["SalePrice","log_SalePrice"], axis=1)

In [507]:
X = pd.get_dummies(X, drop_first=True)

In [508]:
# x_train, x_test, y_train, y_test = train_test_split(X,y, test_size= 0.33)

In [509]:
# xscaler = MinMaxScaler()
# x_train = xscaler.fit_transform(x_train)
# x_test = xscaler.transform(x_test)

In [510]:
lasso = linear_model.Lasso(max_iter = 50000)
trainR2= []
testR2 = []
alphas = np.linspace(1e-4, 1e-3,1000)
for i ,alpha in enumerate(alphas):
    
#     x_train, x_test, y_train, y_test = train_test_split(X,y, test_size= 0.33)
#     xscaler = MinMaxScaler()
#     x_train = xscaler.fit_transform(x_train)
#     x_test = xscaler.transform(x_test)
    x_train_avg = []
    x_test_avg = []
    
    for j in range(3):
        
        x_train, x_test, y_train, y_test = \
        train_test_split(X.to_numpy(),y.to_numpy(), test_size= 0.33)
        
        xscaler = MinMaxScaler()
        x_train = xscaler.fit_transform(x_train)
        x_test = xscaler.transform(x_test)
    
        lasso.set_params(alpha = alpha)
        lasso.fit(x_train, y_train)
        
        x_train_avg.append(lasso.score(x_train, y_train))
        x_test_avg.append(lasso.score(x_test, y_test))

        
    trainR2.append(np.mean(x_train_avg))
    testR2.append(np.mean(x_test_avg))
    print(i, end = '\r')
        
    
best_alpha = alphas[np.argmax(testR2)]
lasso.set_params(alpha = best_alpha)

best_coefs = pd.Series(lasso.coef_, index = X.columns)

    
    

In [516]:
best_alpha

0.00016576576576576575

In [517]:
y = lasso.set_params(alpha = alpha).fit(x_train, y_train)

In [518]:
sorted(list(zip(y.coef_, X.columns)), key=lambda x: abs(x[0]), reverse=True) 

[(0.9492105357646328, 'log_GrLivArea'),
 (0.5294296745208226, 'OverallQual'),
 (0.28392504351343956, 'OverallCond'),
 (0.27593269676453197, 'TotalBsmtSF'),
 (0.1604606899456587, 'YearBuilt'),
 (0.1556623723782854, 'BsmtFinSF1'),
 (0.11639533727233319, 'GarageCars'),
 (0.09987912865209776, 'Neighborhood_StoneBr'),
 (0.08545084003497923, 'GarageArea'),
 (0.08508184225316974, 'Neighborhood_Crawfor'),
 (0.08112954035261993, 'Neighborhood_NridgHt'),
 (0.07259296876304829, 'Num_of_Bath'),
 (0.0647595289599789, 'Neighborhood_NoRidge'),
 (0.06066887891982087, 'YearRemodAdd'),
 (0.05911549348476591, 'Neighborhood_Somerst'),
 (-0.0544140175498774, 'MSZoning_RM'),
 (0.050769908261099284, 'Fireplaces'),
 (-0.04986848753242479, 'MSSubClass'),
 (0.046041566018490014, 'CentralAir_Y'),
 (-0.045160676671483733, 'BldgType_Twnhs'),
 (0.04315645730629596, 'GarageCond_TA'),
 (0.04195197131267815, 'BsmtExposure_Gd'),
 (0.04135291757748472, 'Exterior1st_BrkFace'),
 (-0.039900365934758465, 'KitchenQual_TA'),


In [519]:
max(testR2)

0.9360755501734416

In [515]:
trainR2[np.argmax(testR2)]

0.92537152272284

In [193]:
def calc_vif(data):
    vif = {}
    for feature in data.columns:
        X = [f for f in data.columns if f != feature]
        X = data[X]
        y = data[feature]
        r2 = linear_model.LinearRegression().fit(X,y).score(X,y)
        vif[feature] = 1/(1-r2)
    return pd.DataFrame.from_dict([vif]).T.sort_values(ascending = False, by = 0)

In [223]:
calc_vif(X[sorted_best_coefs[:20]["index"].values[:15]])

,0
square_footage_1,5.718855
GrLivArea,4.300150
OverallQual,3.085383
YearRemodAdd,1.900518
KitchenQual_TA,1.804537
Neighborhood_OldTown,1.759417
Neighborhood_Somerst,1.695000
HeatingQC_Ex,1.495640
GarageType_Attchd,1.480302
BsmtFinType1_GLQ,1.432999


## Random Forest


In [ ]:
### implement random
from sklearn.ensemble import BaggingClassifier
